In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/latimeshealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [3]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,576760256031682561,Sat Mar 14 15:02:15 +0000 2015,Five new running shoes that aim to go the extr...
1,576715414811471872,Sat Mar 14 12:04:04 +0000 2015,Gym Rat: Disq class at Crunch is intense worko...
2,576438353555365888,Fri Mar 13 17:43:07 +0000 2015,Noshing through thousands of ideas at Natural ...
3,576438347003908096,Fri Mar 13 17:43:06 +0000 2015,"Natural Products Expo also explores beauty, su..."
4,576413058177712128,Fri Mar 13 16:02:36 +0000 2015,Free Fitness Weekends in South Bay beach citie...


In [4]:
# -------------------------------------------------
frame.shape

(4171, 3)

In [5]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU'

In [6]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://lat.ms/1ELp3wU
http://lat.ms/1EKOFdr
http://lat.ms/1EHqywg
http://lat.ms/1EHqyfE
http://lat.ms/1EH3SMC
http://lat.ms/1EGNyve
http://lat.ms/1FbQ2kk
http://lat.ms/1b0E0Pi
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1wr2qv0
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1KSN5Kh
http://lat.ms/1AbI1X4
http://lat.ms/1AbI1X4
http://lat.ms/1wm7sZx
http://lat.ms/1BqdRlG
http://lat.ms/1wm7sZx
http://lat.ms/1AbI1X4
http://lat.ms/1wm7sZx
http://lat.ms/1wm7sZx
http://lat.ms/1A84HY3
http://lat.ms/1A84HY3
http://lat.ms/1Gtx9e7
http://lat.ms/1HbYe32
http://lat.ms/1B9Gsg3
http://lat.ms/1GrwjP1
http://lat.ms/1B9Gsg3
http://lat.ms/1GrngO4
http://lat.ms/1McGw0s
http://lat.ms/1H7OGGg
http://lat.ms/1wKRkBh
http://lat.ms/1wKRkBh
http://lat.ms/1McmTr0
http://lat.ms/1EeQS0o
http://lat.ms/1Kv6yR2
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://pbs.twimg.com/media/B
http://lat.ms/17RHiCB
http://lat.ms/1M6N9

http://lati.ms/m8ZBR
http://lati.ms/m99OP
http://lati.ms/m80lg
http://lati.ms/m7ZGG
http://www.latimes.com/news/local/la-me-mta-pesticides-20130617
http://lati.ms/m7sWA
http://lati.ms/m7rOQ
http://lati.ms/m7DR6
http://instagram.com/p/apaep7n4xQ/
http://lati.ms/m70l1
http://lati.ms/m72HF
http://instagram.com/p/aqQAsbhVrR/
http://lat.ms/13LFNye
http://lati.ms/m6ujG
http://lati.ms/m5wR0
http://lati.ms/m5B9s
http://lati.ms/m55Gk
http://lati.ms/m4w9a
http://lati.ms/m32Nb
http://lati.ms/m4vfO
http://instagram.com/p/amTaYdHrGv/
http://lati.ms/m4uXz
http://instagram.com/p/akk1LwPwdd/
http://lati.ms/m3vMW
http://lati.ms/m3pzk
http://lati.ms/m3oQY
http://lati.ms/m36EJ
http://lati.ms/m3vBs
http://lati.ms/m33hr
http://lati.ms/m2V99
http://lati.ms/m3oDu
https://www.strava.com/activities/60257862
http://lati.ms/m2U1N
http://lati.ms/m2DUK
http://lati.ms/m0RYi
http://lati.ms/m1eh0
http://lati.ms/m1f2m
http://lati.ms/m1dGD
http://lati.ms/m19p4
http://lati.ms/m14NY
http://lati.ms/m11fw
http://www.latime

http://lat.ms/yJwUXz
http://lat.ms/wBh2qG
http://lat.ms/wdyaDw
http://lat.ms/zYalIU
http://lat.ms/wizh0z
http://lat.ms/wEOLll
http://lat.ms/wfpE7J
http://lat.ms/ywHBlk
http://lat.ms/yPwYiX
http://lat.ms/wuNWFB
http://lat.ms/xPPchC
http://lat.ms/ArkgmD
http://lat.ms/x43gI5
http://lat.ms/zUFf4G
http://lat.ms/AizPMt
http://lat.ms/wpMg0L
http://lat.ms/yGf0O3
http://lat.ms/w0TrO0
http://lat.ms/wF5EUF
http://lat.ms/wiTsbZ
http://lat.ms/yyErzi
http://lat.ms/AucBJC
http://lat.ms/AuHzoQ
http://lat.ms/zEhrMd
http://lat.ms/zcF04Y
http://lat.ms/wkcc3j
http://lat.ms/zEAmQu
http://lat.ms/x4drxm
http://lat.ms/AsIz4s
http://lat.ms/xw01k1
http://lat.ms/yilRG2
http://lat.ms/zTxBQb
http://lat.ms/xax7Wd
http://lat.ms/ABO88J
http://lat.ms/zFAIZr
http://lat.ms/z9pmuZ
http://lat.ms/z9pmuZ
http://lat.ms/zfklMS
http://lat.ms/yKlJxy
http://lat.ms/zDBVGe
http://lat.ms/zCZKGh
http://lat.ms/yxywHp
http://lat.ms/yqe10C
http://lat.ms/AA2OdS
http://lat.ms/zl8oPp
http://lat.ms/wpe09V
http://lat.ms/zYNRZX
http://lat.ms


http://lat.ms/1yH7haq
Low-income, less educated consumers are less likely to heed calorie postings http://lat.ms/1C6OvKX


http://lat.ms/1C6OvKX
Another benefit of marriage: Your spouse can help you change your life for the better. http://lat.ms/1GrZ7Ia


http://lat.ms/1GrZ7Ia
When something is better than nothing: Calories posted on fast food menus.  http://lat.ms/1C6OvKX


http://lat.ms/1C6OvKX
If this research changes one thing about your pizza-eating habits, it should be to skip the pizza "snack" http://lat.ms/1ynxz0X


http://lat.ms/1ynxz0X
Kids love pizza, but it doesn't love them back. http://lat.ms/1ynxz0X


http://lat.ms/1ynxz0X
You might want to stand up to read this http://lat.ms/1J411L1


http://lat.ms/1J411L1
Even if you exercise, sitting for long periods of time boosts your likelihood of declining health http://lat.ms/1J411L1


http://lat.ms/1J411L1
Hey! Don't just sit there! Worry about how sitting affects your health! http://lat.ms/1J411L1


http://lat.ms/1J411L1
So, h



http://lat.ms/1oWG3HX
Side crunches can make love handles vanish, if done properly http://lat.ms/1oWC6mI


http://lat.ms/1oWC6mI
Robin Williams had Parkinson's disease, his wife said today. Depression can be a symptom of Parkinson's, experts say http://lat.ms/Xl0Nz1


http://lat.ms/Xl0Nz1
Robin Williams had heart disease and depression. It's no coincidence, and more common than you might guess. Scientists are exploring...


A new sleep aid, Belsomra, suppresses the action of the brain's arousal system, a novel approach http://ow.ly/AkENf


http://ow.ly/AkENf
Fewer teens, blacks and Latinas are giving birth outside marriage, but more women over 35 are doing so http://lat.ms/1uuBHuI


http://lat.ms/1uuBHuI
FDA approves Belsomra, the first in a new class of sleep drugs http://lat.ms/1BfiZt9


http://lat.ms/1BfiZt9
Excessive sodium consumption causes some 1.65 million deaths by heart disease and strokes yearly, study says http://lat.ms/1ra27Se


http://lat.ms/1ra27Se
Manufacturer of expe

...

http://lat.ms/19NYgAL
JOIN US: 6:30 a.m. #LATfit chat w/ The Biggest Loser trainer @dolvett on working out with your dog: http://lat.ms/19NYgAL #TOTD


http://lat.ms/19NYgAL
Wanna join #LATFit? It's simple! Just use hashtag to post your healthy choices. Why? We all need the inspiration: http://lati.ms/nJ2Vi


http://lati.ms/nJ2Vi
If you need to resort to fiber pills... it's a sign you're not eating enough veggies! #LATfit #TOTD http://lati.ms/nJ25T


http://lati.ms/nJ25T
No. 17: Band together w/ friends + invest in fun workout DVDs. Swap each month: http://lati.ms/nJ3Ag #LATFit #TOTD ~ @renelynch


http://lati.ms/nJ3Ag
WHOA! MRT @MyLastBite After 7 months of 10,000 DAILY steps (thanks to #LATfit!): For the 1st time in 5 years I didn't gain 1 lb. in 2013


Uh, you are in Antibes, @lauraelizdavis #LATFit nation demands something more scenic than this: http://pbs.twimg.com/media/BQ70qkCCAAEe_fz.jpg


http://pbs.twimg.com/media/BQ70qkCCAAEe
Don't be intimidated by the 5K. Just choose a rac

http://www.latimes.com/news/local/clinic/
Start the work week off right: Here are 17 easy ways to keep active during the day: http://lati.ms/lSlnu #healthtips #LATfit


http://lati.ms/lSlnu
10 tips for keeping a food journal: http://lati.ms/lPiX2 #healthtips #LATfit


http://lati.ms/lPiX2
@citygirlbites Glad we could help!


The kettle bell swing helps develop your hamstrings, glutes and quadriceps – here’s how to do it: http://lati.ms/lPk8O #healthtips #LATfit


http://lati.ms/lPk8O
Not sure which sunscreen to buy this summer? New labeling regulations should help – here’s what they mean: http://lati.ms/lPiae


http://lati.ms/lPiae
Want to look good in that bikini this summer? Try these hip twists to define your waist: http://lati.ms/lI3wn #healthtips #LATfit


http://lati.ms/lI3wn
Follow @mmacvean for live updates from the @BASIPilates conference this weekend. #exercise #pilates #LATfit


Town hall today taught L.A. residents about healthcare coverage – what they asked and what they l



http://lat.ms/XdjRIL
CDC: U.S. not on track to reach 2020 goal for lower smoking rates http://lat.ms/X8b1fq


http://lat.ms/X8b1fq
Let the gym come to you with these fitness sites http://lat.ms/WZnQZj


http://lat.ms/WZnQZj
L.A. becoming more bike friendly, thanks partly to the mayor's elbow http://lat.ms/14gJPk7


http://lat.ms/14gJPk7
Try This: The shoelace, a yin yoga back reliever http://lat.ms/14gJNsn


http://lat.ms/14gJNsn
5 Questions: Byron Hurt seeks a soul food renewal http://lat.ms/14gJMVr


http://lat.ms/14gJMVr
California stem cell agency proposes change after critical report http://lat.ms/14flhrv


http://lat.ms/14flhrv
NAACP catches heat for opposing New York large-soda ban http://lat.ms/WXbwIV


http://lat.ms/WXbwIV
Emergency room visits for ADHD medications rise sharply, report says http://lat.ms/WUVX4w


http://lat.ms/WUVX4w
Gap widens between actual weight and people's imagined weight http://lat.ms/149P7xz


http://lat.ms/149P7xz
Hearing loss, cognitive decline ass

Squeaky clean mice show that skin bacteria help shape immunity http://lat.ms/Nx5SNr


http://lat.ms/Nx5SNr
Fetal DNA tests: Will patents work against patients? http://lat.ms/Q5U1Sq


http://lat.ms/Q5U1Sq
CT for chest pain in ER gets patients home sooner http://lat.ms/MIsxod


http://lat.ms/MIsxod
HIV drug resistance is spreading in Africa, experts say http://lat.ms/LOQaaG


http://lat.ms/LOQaaG
Exposure to light at night may contribute to depression, study says http://lat.ms/NJBU67


http://lat.ms/NJBU67
Heart attack risk balloons after surgery to fix hip or knee http://lat.ms/O8mc6Z


http://lat.ms/O8mc6Z
2012 AIDS meeting: Early treatment is key, experts say http://lat.ms/QAIPlA


http://lat.ms/QAIPlA
LAX: For jump-starting a pandemic, we're No. 2! We're No. 2! http://lat.ms/PWb75f


http://lat.ms/PWb75f
Sally Ride dies of pancreatic cancer: What is pancreatic cancer? http://lat.ms/M7DUYU


http://lat.ms/M7DUYU
New drug combination may shorten, simplify TB therapy http://lat.ms/PU6Y1

Sleep doesn't worsen with age, study shows http://lat.ms/zYNRZX


http://lat.ms/zYNRZX
Kids eat and drink way too much added sugar, the CDC says http://lat.ms/Anxevq


http://lat.ms/Anxevq
Bird flu research: U.S. panel may take another look http://lat.ms/xCvo4s


http://lat.ms/xCvo4s
Producing more babies via automation http://lat.ms/AzzkKU


http://lat.ms/AzzkKU
Lakers' Kobe Bryant sent for MRI post-broken nose: What does it mean? http://lat.ms/AroFnm


http://lat.ms/AroFnm
Sleeping pills linked to higher risk of cancer, death, study says http://lat.ms/xDrLy8


http://lat.ms/xDrLy8
Statins get new safety warnings http://lat.ms/wTwmRZ


http://lat.ms/wTwmRZ
End-of-life care is about much more than pain control http://lat.ms/AcdMGR


http://lat.ms/AcdMGR
Understanding healthcare reform: A picture's worth 1,000 words http://lat.ms/w708Wp


http://lat.ms/w708Wp
Docs' intensive 'get healthy' program whittles the waist -- a little http://lat.ms/w78J7Y


http://lat.ms/w78J7Y
Eggs made from s

In [7]:
frame.head()

,tweetid,date,title,url
0,576760256031682561,Sat Mar 14 15:02:15 +0000 2015,Five new running shoes that aim to go the extr...,http://lat.ms/1ELp3wU
1,576715414811471872,Sat Mar 14 12:04:04 +0000 2015,Gym Rat: Disq class at Crunch is intense worko...,http://lat.ms/1EKOFdr
2,576438353555365888,Fri Mar 13 17:43:07 +0000 2015,Noshing through thousands of ideas at Natural ...,http://lat.ms/1EHqywg
3,576438347003908096,Fri Mar 13 17:43:06 +0000 2015,"Natural Products Expo also explores beauty, su...",http://lat.ms/1EHqyfE
4,576413058177712128,Fri Mar 13 16:02:36 +0000 2015,Free Fitness Weekends in South Bay beach citie...,http://lat.ms/1EH3SMC


In [7]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4171 entries, 0 to 4170
Data columns (total 4 columns):
tweetid    4171 non-null int64
date       4171 non-null object
title      4171 non-null object
url        4171 non-null object
dtypes: int64(1), object(3)
memory usage: 130.4+ KB


In [8]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [5]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class=" card-content "]/p/text()',
               '',
               '']



In [14]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class=" card-content "]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@class="byline-wrapper"]/span/a/span/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://lat.ms/1ELp3wU
5405
11
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  5405
tags  :  0
author  :  11




scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://lat.ms/1EKOFdr
2135
0
contents  :  2135
tags  :  0
author  :  0


Header:   200
http://lat.ms/1EHqywg
5918
12
contents  :  5918
tags  :  0
author  :  12


Header:   200
http://lat.ms/1EHqyfE
3079
0
contents  :  3079
tags  :  0
author  :  0


Header:   200
http://lat.ms/1EH3SMC
2056
12
contents  :  2056
tags  :  0
author  :  12


Header:   200
http://lat.ms/1EGNyve
2776
0
contents  :  2776
tags  :  0
author  :  0


Header:   200
http://lat.ms/1FbQ2kk
4668
13
contents  :  4668
tags  :  0
author  :  13


Header:   200
http://lat.ms/1b0E0Pi
4041
13
contents  :  4041
tags  :  0
author  :  13


Header:   200
http://lat.ms/1KSN5Kh
3502
12
contents  :  3502
tags  :  0
author  :  12


Header:   200
http://lat.ms/1KSN5Kh
3502
12
contents  :  3502
tags  :  0
author  :  12


Header:   200
http://lat.ms/1wr2qv0
4372
11
contents  :  4372
tags  :  0
author  :  11


Header:   200
http://lat.ms/1KSN5Kh
3502
12
contents  :  3502
tags  :  0
author  :  12


Header:   200
http://lat.ms

Header:   200
http://lat.ms/1CoY5HM
3570
0
contents  :  3570
tags  :  0
author  :  0


Header:   200
http://lat.ms/1zAY4fj
4323
13
contents  :  4323
tags  :  0
author  :  13


Header:   200
http://lat.ms/1zAY4fj
4323
13
contents  :  4323
tags  :  0
author  :  13


Header:   200
http://lat.ms/1Co84gE
4698
11
contents  :  4698
tags  :  0
author  :  11


Header:   200
http://lat.ms/1CjU6w3
5047
11
contents  :  5047
tags  :  0
author  :  11


Header:   200
http://lat.ms/1CiC3nt
3791
12
contents  :  3791
tags  :  0
author  :  12


Header:   200
http://lat.ms/1CjU6w3
5047
11
contents  :  5047
tags  :  0
author  :  11


Header:   200
http://lat.ms/1CiC3nt
3791
12
contents  :  3791
tags  :  0
author  :  12


Header:   200
http://lat.ms/1E4QxKi
5982
13
contents  :  5982
tags  :  0
author  :  13


Header:   200
http://lat.ms/1yhOMEP
3124
13
contents  :  3124
tags  :  0
author  :  13


Header:   200
http://lat.ms/16SLUYm
3893
12
contents  :  3893
tags  :  0
author  :  12


Header:   200
http://la

Header:   200
http://lat.ms/1ynxz0X
2352
12
contents  :  2352
tags  :  0
author  :  12


Header:   200
http://lat.ms/1ynxz0X
2352
12
contents  :  2352
tags  :  0
author  :  12


Header:   200
http://lat.ms/1J411L1
4370
13
contents  :  4370
tags  :  0
author  :  13


Header:   200
http://lat.ms/1J411L1
4370
13
contents  :  4370
tags  :  0
author  :  13


Header:   200
http://lat.ms/1J411L1
4370
13
contents  :  4370
tags  :  0
author  :  13


Header:   200
http://lat.ms/1Glbcii
4288
12
contents  :  4288
tags  :  0
author  :  12


Header:   200
http://lat.ms/1Bhbkwd
4314
0
contents  :  4314
tags  :  0
author  :  0


Header:   200
http://lat.ms/1E559K1
3201
13
contents  :  3201
tags  :  0
author  :  13


Header:   200
http://lat.ms/1xtH0qC
5099
13
contents  :  5099
tags  :  0
author  :  13


Header:   200
http://lat.ms/1x6uC0X
1237
12
contents  :  1237
tags  :  0
author  :  12


Header:   200
http://lat.ms/1x6uC0X
1237
12
contents  :  1237
tags  :  0
author  :  12


Header:   200
http://la

...

Header:   200
http://lat.ms/1o5uQWp
2597
0
contents  :  2597
tags  :  0
author  :  0


Header:   200
http://lat.ms/1eWEODv
2242
0
contents  :  2242
tags  :  0
author  :  0


Header:   200
http://lat.ms/1iycPL0
3175
0
contents  :  3175
tags  :  0
author  :  0


Header:   200
http://lat.ms/1iycPL0
3175
0
contents  :  3175
tags  :  0
author  :  0


Header:   200
http://lat.ms/1fWS47a
1094
0
contents  :  1094
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dsIT46
2742
0
contents  :  2742
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dsIT46
2742
0
contents  :  2742
tags  :  0
author  :  0


Header:   200
http://lat.ms/1hVFBDj
5676
0
contents  :  5676
tags  :  0
author  :  0


Header:   200
http://lat.ms/1rqxJQQ
2603
0
contents  :  2603
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dqwDB2
3143
0
contents  :  3143
tags  :  0
author  :  0


Header:   200
http://lat.ms/1jx37rq
2734
0
contents  :  2734
tags  :  0
author  :  0


Header:   200
http://instagram.com/p/l6Oulj

Header:   200
http://lat.ms/1eO4xJq
2333
0
contents  :  2333
tags  :  0
author  :  0


Header:   200
http://lat.ms/ManL57
4772
0
contents  :  4772
tags  :  0
author  :  0


Header:   200
http://lat.ms/1eJnZan
3990
0
contents  :  3990
tags  :  0
author  :  0


Header:   200
http://lat.ms/1kI2Oxy
2245
0
contents  :  2245
tags  :  0
author  :  0


['ERRORS'] 404 , at http://www.latimes.com/local/lanow/la-me-ln-flu-deaths-california-20140214
Header:   200
http://lat.ms/1ovdq3i
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://lat.ms/NHEnD8
4393
0
contents  :  4393
tags  :  0
author  :  0


Header:   200
http://lat.ms/NHEnD8
4393
0
contents  :  4393
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dLrE7g
2853
0
contents  :  2853
tags  :  0
author  :  0


Header:   200
http://lat.ms/1kF91dJ
4464
0
contents  :  4464
tags  :  0
author  :  0


Header:   200
http://lat.ms/MM8nMV
4063
0
contents  :  4063
tags  :  0
author  :  0


Header:   200
http://lat.ms/MM8nMV
4063
0
con

Header:   200
http://lat.ms/1dz2rRx
4294
0
contents  :  4294
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dz2rRx
4294
0
contents  :  4294
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dz2rRx
4294
0
contents  :  4294
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dbGrrp
3365
0
contents  :  3365
tags  :  0
author  :  0


Header:   200
http://lat.ms/Km8DRq
4939
0
contents  :  4939
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dzhWKH
5080
0
contents  :  5080
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dgtdNl
5212
0
contents  :  5212
tags  :  0
author  :  0


Header:   200
http://lat.ms/1eKTKDU
2575
0
contents  :  2575
tags  :  0
author  :  0


Header:   200
http://lat.ms/1eKTKDU
2575
0
contents  :  2575
tags  :  0
author  :  0


Header:   200
http://lat.ms/1kCNBio
4501
0
contents  :  4501
tags  :  0
author  :  0


Header:   200
http://lat.ms/1hrK9SU
4232
0
contents  :  4232
tags  :  0
author  :  0


Header:   200
http://lat.ms/1dthT1E
3022
0
c

Header:   200
http://lat.ms/1891Q9i
4236
0
contents  :  4236
tags  :  0
author  :  0


Header:   200
http://lat.ms/18QH1d9
3851
0
contents  :  3851
tags  :  0
author  :  0


Header:   200
http://lat.ms/1891Q9i
4236
0
contents  :  4236
tags  :  0
author  :  0


Header:   200
http://lat.ms/Igsi4g
2589
0
contents  :  2589
tags  :  0
author  :  0


Header:   200
http://lat.ms/Igsi4g
2589
0
contents  :  2589
tags  :  0
author  :  0


Header:   200
http://lat.ms/19Ewcdw
4773
0
contents  :  4773
tags  :  0
author  :  0


Header:   200
http://lat.ms/1h0PuCE
3217
0
contents  :  3217
tags  :  0
author  :  0


Header:   200
http://lat.ms/19D88b8
5830
0
contents  :  5830
tags  :  0
author  :  0


Header:   200
http://lat.ms/19D88b8
5830
0
contents  :  5830
tags  :  0
author  :  0


Header:   200
http://lat.ms/1fKJn2h
6797
0
contents  :  6797
tags  :  0
author  :  0


Header:   200
http://lat.ms/1igIhgZ
3765
0
contents  :  3765
tags  :  0
author  :  0


Header:   200
http://lat.ms/1auZ7o8
4043
0
co

Header:   200
http://lat.ms/1hFNifR
3098
0
contents  :  3098
tags  :  0
author  :  0


Header:   200
http://lat.ms/181Dtr8
2817
0
contents  :  2817
tags  :  0
author  :  0


Header:   200
http://lat.ms/1bgcGXN
2640
0
contents  :  2640
tags  :  0
author  :  0


Header:   200
http://lat.ms/1cAuf8P
2434
0
contents  :  2434
tags  :  0
author  :  0


['ERRORS'] 404 , at http://lat.ms/16zoWTD
Header:   200
http://lat.ms/15TSZH9
2907
0
contents  :  2907
tags  :  0
author  :  0


Header:   200
http://lat.ms/15TSZH9
2907
0
contents  :  2907
tags  :  0
author  :  0


Header:   200
http://lat.ms/GJwl8r
4149
0
contents  :  4149
tags  :  0
author  :  0


Header:   200
http://lat.ms/1coBaC7
3966
0
contents  :  3966
tags  :  0
author  :  0


Header:   200
http://lat.ms/1b3YKA3
6056
0
contents  :  6056
tags  :  0
author  :  0


['ERRORS'] 404 , at http://pbs.twimg.com/media/BV09rtyCUAAq
Header:   200
http://lat.ms/16KeV8m
2144
0
contents  :  2144
tags  :  0
author  :  0


Header:   200
http://lat.ms/G

In [15]:
# -------------------------------------------------
frame.head()

,author,contents,date,tags,title,tweetid,url
0,Roy Wallack,A day before the 30th anniversary of the L.A. ...,Sat Mar 14 15:02:15 +0000 2015,,Five new running shoes that aim to go the extr...,5.767603e+17,http://lat.ms/1ELp3wU
1,,Crunch prides itself on offering oddball group...,Sat Mar 14 12:04:04 +0000 2015,,Gym Rat: Disq class at Crunch is intense worko...,5.767154e+17,http://lat.ms/1EKOFdr
2,Mary MacVean,"After tasting our way, along with 70,000 other...",Fri Mar 13 17:43:07 +0000 2015,,Noshing through thousands of ideas at Natural ...,5.764384e+17,http://lat.ms/1EHqywg
3,,The Natural Products Expo West included more t...,Fri Mar 13 17:43:06 +0000 2015,,"Natural Products Expo also explores beauty, su...",5.764383e+17,http://lat.ms/1EHqyfE
4,Mary MacVean,What keeps you from trying that workout class?...,Fri Mar 13 16:02:36 +0000 2015,,Free Fitness Weekends in South Bay beach citie...,5.764131e+17,http://lat.ms/1EH3SMC


In [16]:
# -------------------------------------------------
frame.shape

(4171, 7)

In [17]:
frame.to_csv('./run_data/latimes_scrapt_v2.csv', sep='|', encoding='utf-8',index=False)

In [18]:
test_url= 'http://lat.ms/1EGNyve'

res = requests.get(test_url)
if res.status_code  == 200:
#     soup = BeautifulSoup(res.content, 'lxml')
#     author = soup.find("span",{"class":"byline"})
#     print(author)
#     content = soup.find("div", {"class": "article-body"})
#     print(content)

    # Initialize XPath
    xpath_selector = Selector(text=res.text)
    
#     list_scrapt = scrapt_w_xpath(list_scrapt)
#     print("list_Scrapt: ",list_scrapt)
    content = xpath_selector.xpath('//div[@class=" card-content "]/p/text()').extract()
    content = join_txt(content)
    print(content)
else:
    print("[ERROR]: ", res.status_code)

When that Olympic gold medal for judo is hung around your neck, you know you're the best in the world. Sometimes Kayla Harrison, 24, hated the endless training, missing movie nights and even the prom, but having the ability to visualize her success coupled with a clear objective to be the champion allowed her to persevere in London 2012, and perhaps again in Rio de Janeiro in 2016.. I started when I was 6 because my mother had taken judo in college and she wanted me to learn self-defense. She enrolled me in the local dojo, and I just really enjoyed it. Before that I played a little bit of T-ball and did a lot of dance, but from 6 years old onward, judo was my passion.. The better I got, the more I had to sacrifice other parts of my life. I started doing school online when I was 16 and moved to Boston to train with the Olympic coaches. I never went to prom, and most nights when my friends were going to the movies I'd be working out at the club.. It is all-consuming. It's something that 

In [13]:
title_attempt = True
for key, url in enumerate(['http://lat.ms/1EGNyve','http://lat.ms/1b0E0Pi']):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class=" card-content "]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@class="byline-wrapper"]/span/a/span/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)
                
                title_attempt = False
#             print(list_content)
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))    
#                 print(frame.loc[key,k_])
                frame.loc[key,k_] = v_
            
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://lat.ms/1EGNyve
2776
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  2776
tags  :  0
author  :  0




scripts.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  def get_hastags_out(regex, sample):


Header:   200
http://lat.ms/1b0E0Pi
4041
13
contents  :  4041
tags  :  0
author  :  13




In [34]:
frame.head()

,author,contents,date,tags,title,tweetid,url
0,NaN,NaN,Sat Mar 14 15:02:15 +0000 2015,NaN,Five new running shoes that aim to go the extr...,5.767603e+17,http://lat.ms/1ELp3wU
1,NaN,NaN,Sat Mar 14 12:04:04 +0000 2015,NaN,Gym Rat: Disq class at Crunch is intense worko...,5.767154e+17,http://lat.ms/1EKOFdr
2,NaN,NaN,Fri Mar 13 17:43:07 +0000 2015,NaN,Noshing through thousands of ideas at Natural ...,5.764384e+17,http://lat.ms/1EHqywg
3,NaN,NaN,Fri Mar 13 17:43:06 +0000 2015,NaN,"Natural Products Expo also explores beauty, su...",5.764383e+17,http://lat.ms/1EHqyfE
4,NaN,NaN,Fri Mar 13 16:02:36 +0000 2015,NaN,Free Fitness Weekends in South Bay beach citie...,5.764131e+17,http://lat.ms/1EH3SMC


In [14]:
df_temp.head()

NameError: name 'df_temp' is not defined